In [7]:
# 라이브러리 호출
import numpy as np
import scipy.io
import matplotlib.pyplot as plt

In [13]:
# MAT 파일 불러오기
mat_path = r"../HW3_package/hw3_prob1a.mat"
mat_data = scipy.io.loadmat(mat_path)

# 변수 할당
A = mat_data['A']
b = mat_data['b']
x_orig = mat_data['x_orig']

In [15]:
def g(x):
    return (np.linalg.norm((A @ x - b), 2) ** 2)

def h(x):
    return lambda_v * np.linalg.norm((A @ x - b), 1)

def f(x):
    return (g(x) + h(x))/2

In [16]:
def grad_g(x):
    return A.T @ (A @ x - b)

In [ ]:
# ISTA
x = np.zeros((A[1], 1))

lambda_v = 2
# t = 1/L 
t =  np.linalg.norm(A, 2) ** 2
stopping_point = 1e-4


while True:
    
    delta_x = x - t * grad_g(xk)
   
    x = np.where(np.abs(temp_x) > TH, (np.abs(temp_x) - TH) * np.sign(temp_x), 0)
    
    # break check 
    if  (np.linalg.norm((xk - xk_1), 2)/np.linalg.norm(xk, 1))  > stopping_point:
        break
    xk_old = xk
    
    